# Solution Seekers Group

Lead of the Study Group Discussion: **Badr Bensassi**

Author: **Youssef Laouina**


> **“All Models are wrong, but some are useful.”**

*George Box*

Importing our libraries

In [ ]:
# !pip install category_encoders

In [ ]:
import warnings

import pandas as pd
import numpy as np
from scipy.stats import pearsonr 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline

from category_encoders import OneHotEncoder

warnings.simplefilter(action="ignore", category=FutureWarning)

# Prepare Data

## Importing our data into a Pandas DataFrame

In [ ]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/tirthajyoti/Machine-Learning-with-Python/master/Datasets/USA_Housing.csv', index_col=False)

In [ ]:
df.info()

In [ ]:
df.head()

## Clean our data

### Clean feature names

In [ ]:
new_cols = df.columns.str.replace('.', '').str.replace(' ', '_')

In [ ]:
df.columns = new_cols

In [ ]:
df.info()

### Drop irrelevant observations

In [ ]:
pd.options.display.max_colwidth = 200

In [ ]:
df['Address'].head(10)

We will drop records that contains ***?PO*** as they don't align with our model's target area, potentially introducing irrelevant information.

* **?PO**: Stands for "Post Office," which is used for U.S. Navy, Army installations stationed overseas, or Coast Guard installations that are deployed or stationed overseas.



In [ ]:
df[df['Address'].str.contains('PO') == True]['Address']

In [ ]:
index_drop = df[(df['Address'].str.contains('PO') == True)]['Address'].index.tolist()

In [ ]:
df.drop(index_drop, axis=0, inplace=True)

### Extract City & State Code

Now we will extract the **city** and **state** code from `Address`

In [ ]:
df['city-state'] = df['Address'].str.split('\n', expand=True)[1]

In [ ]:
df['city'] = df['city-state'].str.split(', ', expand=True)[0]

df['state_code'] = df['city-state'].str.split(', ', expand=True)[1].str[:2]

In [ ]:
df.info()

### Drop irrelevant columns

In [ ]:
df.columns

In [ ]:
df.drop(columns=['Address', 'city-state'], inplace=True)

In [ ]:
df.info()

In [ ]:
df.head()

# Build Model

In [ ]:
feature_matrix = df.drop(columns=['Price', 'city'])
target_vector = df['Price']

print(f"Feature Matrix: {feature_matrix.shape}",
      f"Target Vector: {target_vector.shape}",
      sep='\n')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature_matrix,
                                                    target_vector,
                                                    train_size=0.8,
                                                    random_state=7)

## Baseline Model

In [ ]:
y_mean = y_train.mean()
y_pred_baseline = [y_mean] * len(y_train)
mse_baseline = mean_squared_error(y_train, y_pred_baseline)

print("Mean house price:", round(y_mean, 2))

print("Baseline RMSE:", round(np.sqrt(mse_baseline), 2))

## Iterate

If we try to fit a LinearRegression predictor to your training data at this point, we'll get an error that looks like this:

`ValueError: could not convert string to float`

What does this mean? When we fit a linear regression model, we're asking scikit-learn to perform a mathematical operation. The problem is that our training set contains city and state information in non-numerical form. In order to create our model we need to **encode** that information so that it's represented numerically.

The good news is that there are lots of transformers that can do this. Here, we'll use the one from the ***Category Encoders*** library, called a **OneHotEncoder**.

Before we build include this transformer in our model, let's explore how it works.

### Encode non-numerical features

In [ ]:
ohe = OneHotEncoder(use_cat_names=True)
ohe.fit(X_train)

XT_train = ohe.transform(X_train)

print(XT_train.shape)
XT_train.head()

### Make pipeline

Now we will create a pipeline named model that contains a OneHotEncoder transformer and a LinearRegression predictor. Then fit our model to the training data.

In [ ]:
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    LinearRegression()
)

model.fit(X_train, y_train)

In [ ]:
y_pred_lm_train = model.predict(X_train)
y_pred_lm_test = model.predict(X_test)

## Evalute

### Performance Metrics

In [ ]:
mse_lm_train = mean_squared_error(y_train, y_pred_lm_train)
mse_lm_test = mean_squared_error(y_test, y_pred_lm_test)

print("MLR metrics: ")
print("\tTraining MSE:\t", round(mse_lm_train, 2))
print("\tTraining RMSE:\t", round(np.sqrt(mse_lm_train), 2))
print()
print("\tTesting MSE:\t", round(mse_lm_test, 2))
print("\tTesting RMSE:\t", round(np.sqrt(mse_lm_test), 2))
print()
print("\tR-squared:\t", round(r2_score(y_true=y_test, y_pred=y_pred_lm_test), 2))

### Feature importance

In [ ]:
intercept = model.named_steps['linearregression'].intercept_
coefficients = model.named_steps['linearregression'].coef_
print("Number of coefficients:", len(coefficients))
print("Coefficients: ", coefficients)

In [ ]:
feature_names = model.named_steps['onehotencoder'].get_feature_names()
print("Features len:", len(feature_names))
print(feature_names)

We will create a pandas Series named `feat_imp` where the index is our `features` and the values are our `coefficients`.


In [ ]:
feat_imp = pd.Series(coefficients ,index=feature_names)
feat_imp.head(10)

Now we will create a horizontal bar chart that shows top 15 coefficients for our model.

In [ ]:
feat_imp.sort_values(key=np.abs).tail(15).plot(kind='barh')
plt.xlabel('Importance [USD]')
plt.ylabel('Features/Predictors')
plt.title('Feature Importance for House Price')
plt.show()

Looking at this bar chart, we can see that `Avg. Area House Age` and `Avg. Area Number of Rooms` are the two features having the greater weights importance, while the other features have a small effect on the prediction power of our model.

The analysis of **Feature Importance** can help us determine the most influential features in our model and possibly reduce the complexity of our model, but it is not enough to conclude if a feature has a significant effect on our model or not.

We now have 64 predictor variables to choose from, so we need a way of guiding us to choose the best ones to be our predictors.

One way is to look at the correlations between the `Price` and each variable in our dataset and select those with the strongest correlations - both positive and negative.

We also need to consider how significant those features are. 

# Feature selection

## By correlation:

Prepare our dataframe with the encoded state codes:

In [ ]:
feature_matrix_all = df.drop(columns='city')

In [ ]:
ohe = OneHotEncoder(use_cat_names=True)
ohe.fit(feature_matrix_all)

df_encoded = ohe.transform(feature_matrix_all)

print(df_encoded.shape)
df_encoded.head()

In [ ]:
corr = df_encoded.corr()['Price']

correlation_df = pd.DataFrame(corr, index=corr.index).sort_values(by='Price', ascending=False)

In [ ]:
correlation_df

In [ ]:
correlation_df = correlation_df.drop('Price')

In [ ]:
correlation_df

### Create and visualise the correlation matrix

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))

sns.barplot(y=correlation_df.index ,x=correlation_df['Price'], ax=ax, orient='h')

plt.show()

In [ ]:
column_titles = corr.index.tolist()
column_titles.remove('Price')

In [ ]:
# Build a dictionary of correlation coefficibinents and p-values
dict_cp = {}

for col in column_titles:
    p_val = pearsonr(df_encoded[col], df_encoded['Price'])[1]
    
    dict_cp[col] = {
        'Correlation_Coefficient': correlation_df.loc[col].values[0],
        'P_Value': p_val
    }

df_cp = pd.DataFrame(dict_cp).T

df_cp_sorted = df_cp.sort_values(by='P_Value')

df_cp_sorted

In [ ]:
significance_level = 0.05

df_cp_sorted[df_cp_sorted['P_Value'] <= significance_level]

> At a significance level of 5%, we would filter our predictors to only include 10 features instead of 64.

### Understanding the p-value in correlation analysis

The p-value is a statistical measure that helps assess the significance of the correlation observed in a dataset. It quantifies the probability of obtaining a correlation as extreme as the one observed, assuming that the variables are actually uncorrelated (null hypothesis). Here's how the calculation generally proceeds:

1. **Compute the Pearson correlation coefficient:** This measures the linear relationship between two variables, ranging from -1 to 1.

2. **Assume the null hypothesis:** The null hypothesis states that there is no significant correlation between the variables in the population.

3. **Generate a distribution of correlations under the null hypothesis (e.g., using bootstrapping):** This involves simulating many datasets with the same size as the original dataset but with uncorrelated variables. The Pearson correlation coefficient is calculated for each simulated dataset.

4. **Calculate the p-value using the Cumulative Distribution Function (CDF):** The p-value is determined by comparing the observed correlation coefficient from the original dataset with the distribution of correlation coefficients generated under the null hypothesis. It represents the probability of obtaining a correlation coefficient at least as extreme as the observed one, assuming the null hypothesis is true.

5. **Interpretation:** A small p-value (typically less than 0.05) suggests that the observed correlation is unlikely to have occurred by chance alone, leading to the rejection of the null hypothesis and indicating a statistically significant correlation.

So, the p-value helps quantify the likelihood that the observed correlation in the dataset is due to random chance versus a true underlying relationship between the variables.


## By variance:

Variance Thresholds remove features whose values don't change much from observation to observation.

The objective here is to remove all features that have a variance lower than the selected threshold.

**Note:** Variance is dependent on scale, so the features will have to be ***normalized*** before implementing variance thresholding.

### Normalize data

In [ ]:
# Separate data into independent (X) and dependent (y) variables

X_data = df_encoded.drop(columns='Price')
y_data = df_encoded['Price']

In [ ]:
# Normalize data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X_data)
X_normalized = pd.DataFrame(X_scaled, columns=X_data.columns)

In [ ]:
X_normalized.head()

### Perform variance selection

Variance Threshold in Scikit Learn

To implement Variance Threshold in Scikit Learn we have to do the following:

* Import and create an instance of the VarianceThreshold class
* Use the .fit() method to select subset of features based on the threshold.


In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
# Create VarianceThreshold object
selector = VarianceThreshold(threshold=0.01879)

# Use the object to apply the threshold on data
selector.fit(X_normalized)

The Variance Threshold has been applied to the data. Let's look at the calculated variance for each predictive variable:

In [ ]:
# Get column variances
column_variances = selector.variances_

vars_dict = [
    {
        "Variable_Name": c_name,
        "Normalized_Variance": c_var
    }
    for c_name, c_var in zip(X_normalized.columns, column_variances)
]

df_vars = pd.DataFrame(vars_dict)
df_vars.sort_values(by='Normalized_Variance', ascending=False)

The above table shows the variances of the individual columns before any threshold is applied. It allows us to **revise our initial variance threshold** if we feel that we might exclude important variables.


Next we need to **extract the results** and use them to **select our new columns** - which form a subset of all the columns:

In [ ]:
# Select columns
X_selected = X_normalized[X_normalized.columns[selector.get_support(indices=True)]]

# Save variable names for later
X_var_names = X_selected.columns

# View first few entries
X_selected.columns

#### Treashold Tunning

In [ ]:
treashold_range = np.linspace(start=0, stop=0.03, num=100)

In [ ]:
# tunning of the treashold:

for i in treashold_range:
    treashold = np.round(i, 5)
    selector = VarianceThreshold(threshold=treashold)
    
    selector.fit(X_normalized)
    feat_num = selector.get_support(indices=True).shape[0]

    print(f"With threshold = {treashold}:\n\tNumber of features is {feat_num}\n")

    if feat_num == 1:
        break

~ Y.L